In [ ]:
###GSA Inventory tackle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns


In [ ]:
solution_finder_data = pd.read_csv("C:/Users/dkim/Downloads/solution_finder_data_grouped.csv")
print(solution_finder_data.shape)
solution_finder_data = solution_finder_data.rename(columns={'  "Title"': 'Solution Name',
                                                            'CAS Agency': 'Managing Agency'})

In [ ]:
contract_inventory_testing = pd.read_csv("C:/Users/dkim/Downloads/contract_inventory_data_grouped.csv")

In [ ]:
####Fixing the Managing Agency name
replace_dict = {'name1': 'DOD', 'name2': 'DOJ'}

solution_finder_data['Managing Agency'] = solution_finder_data['Managing Agency'].replace(replace_dict)

In [ ]:
solution_finder_data['Managing Agency'].unique()

In [ ]:
contract_inventory_testing['Managing Agency'].unique()

In [ ]:
##For contract inventory managing agency lets create a second one column where the managing agency is mixed together by the bigger agency
contract_inventory_testing['Managing Agency modified'] = contract_inventory_testing['Managing Agency']

replace_dict = {'1': 'DOD', '2':'DOD', '3':'DOD','4':'DOD','5':'DOD','6':'DOD', '7': 'DHHS','8':'DHHS','9':'DOD','10':'DVA'}
contract_inventory_testing['Managing Agency modified'] = contract_inventory_testing['Managing Agency modified'].replace(replace_dict)

In [ ]:
from thefuzz import fuzz
def fuzzy_match(row): #result: Its better when we modified the managing agency before hand, however we need to look over the gsa contract
#    data_sub = solution_finder_data_GSA[solution_finder_data_GSA['Managing Agency'] == row['Managing Agency modified']]

    
    token_set_ratio = (solution_finder_data_GSA['Solution Name']
                       .apply(lambda y: fuzz.token_set_ratio(row['Solution Name'], y)) )
    
    # Create a boolean list in which the highest score is selected
    max_row         = (token_set_ratio == token_set_ratio.max())
    
    # Return the highest score and the highest solution name
    #return token_set_ratio[max_row].iloc[0], solution_finder_data.loc[max_row, 'Solution Name'].iloc[0] #original
    return token_set_ratio[max_row].iloc[0], solution_finder_data_GSA.loc[max_row, 'Solution Name'].iloc[0], solution_finder_data_GSA.loc[max_row, 'Managing Agency'].iloc[0]

# Return the highest score and highest solution name as a list
x = contract_inventory_testing.apply(fuzzy_match, 1)

# Map the first item in the list to a new column "Similarity Score"
contract_inventory_testing['Similarity Score'] = [v[0] for v in x]

# Map the second item in the list to a new column "Solution Map"
contract_inventory_testing['Solution Map']     = [v[1] for v in x]

# Map the third item in the list to a new column "Solution agency"
contract_inventory_testing['Solution Agency']     = [v[2] for v in x] #extra line, unknown 

In [ ]:
managing_agency_counts = solution_finder_data['Managing Agency'].value_counts(normalize=True) * 100

# The result is a Series with the Managing Agency categories as the index and the percentages as the values
print(managing_agency_counts)


In [ ]:
###Summary stats for the metdadata (08/09/2023)
summary_stats = pd.read_csv("C:/Users/dkim/Downloads/20230803 Solutions Metadata.csv")
summary_stats

In [ ]:
summary_stats.columns
summary_stats_only_matched = summary_stats[(summary_stats['Found in CI'] == 1) & (summary_stats['Found in SF'] == 1)]
summary_stats_only_matched

In [ ]:
number_of_unique_values = summary_stats_only_matched['Contract Inventory Name'].nunique()

print(number_of_unique_values) #239 unique contract name or 5% of contract inventory data


In [ ]:
# Dropping duplicate rows based on the "Contract Inventory Name" column #This will find the number of unique contracts in contract inventory name 
unique_contracts_df = summary_stats_only_matched.drop_duplicates(subset=['Contract Inventory Name','Managing Agency (Contract inventory)'])

# Calculating the percentage distribution for the "Managing Agency (Contract inventory)" column
managing_agency_counts = unique_contracts_df['Managing Agency (Contract inventory)'].value_counts(normalize=True) * 100

# Printing the results
print(managing_agency_counts)

managing_agency_counts = unique_contracts_df['Managing Agency (Contract inventory)'].value_counts() 
print(managing_agency_counts)

import matplotlib.pyplot as plt

# Assuming managing_agency_counts is already created as shown in your code
managing_agency_counts = unique_contracts_df['Managing Agency (Contract inventory)'].value_counts(normalize=True) * 100

# Identify categories with less than 4% and combine them into "Other"
other_percentage = managing_agency_counts[managing_agency_counts < 4].sum()
managing_agency_counts = managing_agency_counts[managing_agency_counts >= 4]
managing_agency_counts['Other'] = other_percentage

# Creating a pie chart
plt.figure(figsize=(9, 10))
plt.pie(managing_agency_counts, labels=managing_agency_counts.index, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures the pie chart is circular
plt.title('Solution Finders Managing Agency Categories')
plt.show()